In [1]:
import pandas as pd
from core.util.metrics import mae, rmse, smape
from core.util.naive_baselines import naive, _24hlag
from core.util.get_datasets import split_data, denormalize_data
from core.util.hyperparameter_configuration import get_hyperparameter_configuration

In [2]:
hyperparameters = get_hyperparameter_configuration()
loss_function = hyperparameters["loss_function"]
horizon = hyperparameters["horizon"]
train_days = hyperparameters["train_days"]
val_days = hyperparameters["val_days"]
test_days = hyperparameters["test_days"]
lookback = 96

In [3]:
_, _, _, _, x_test, y_test = split_data(
    lookback=lookback,
    horizon=horizon,
    train_days=train_days,
    val_days=val_days,
    test_days=test_days,
    features={},
    park_nums=[4],
)

x_test = denormalize_data(x_test)
y_test = denormalize_data(y_test)

In [4]:
naive_prediction = naive(x_test)
lagged_prediction = _24hlag(x_test)

y_test = y_test.flatten()
naive_prediction = naive_prediction.flatten()
lagged_prediction = lagged_prediction.flatten()

In [5]:
loss = {
    "NBE": [
        mae(y_test, naive_prediction).item(),
        rmse(y_test, naive_prediction),
        smape(y_test, naive_prediction).item(),
    ],
    "24hNBE": [
        mae(y_test, lagged_prediction).item(),
        rmse(y_test, lagged_prediction),
        smape(y_test, lagged_prediction).item(),
    ],
}

pd.DataFrame(loss, index=["MAE", "RMSE", "sMAPE"])

,NBE,24hNBE
MAE,21.213496,18.803129
RMSE,41.641587,49.864295
sMAPE,88.984226,43.620495
